In [23]:
# load csv
import pandas as pd
csv_path = r"https://docs.google.com/spreadsheets/d/1EvIFLGpsKvuOgdX6F9MYlGl9rCLjbKHPlE8srBDMwYo/edit?usp=sharing"
url = csv_path.replace('/edit?usp=sharing', '/export?format=xlsx')

df = pd.read_excel(url, sheet_name="iPad")

df.columns

Index(['id', 'date', 'writer', 'model', 'screen_size', 'generation', 'ram',
       'storage', 'cellular', 'apple_care_plus', 'unused', 'price',
       'price for statistics', 'platform', 'url', 'title', 'content'],
      dtype='object')

In [24]:
import requests
item_csv_path = r"https://docs.google.com/spreadsheets/d/1eRkhdeDaD60IOajReYVRId28CM4IKQJiheL-yBWfzUw/edit?usp=sharing"
url = item_csv_path.replace('/edit?usp=sharing', '/export?format=xlsx')


item_df = pd.read_excel(url, sheet_name="iPad-Docs")

api_db = {}

for index, row in item_df.iterrows():
    value = (row["item id"], row["option id"])
    key = (row["item"], row["gen"], row["cellular"], row["storage"])
    print(key)
    api_db[key] = value


('iPad Mini', '6세대', 'Wi-Fi', '64GB')
('iPad Mini', '6세대', 'Wi-Fi', '256GB')
('iPad Mini', '6세대', 'Wi-Fi+Cellular', '64GB')
('iPad Mini', '6세대', 'Wi-Fi+Cellular', '256GB')
('iPad Air', '4세대', 'Wi-Fi', '64GB')
('iPad Air', '4세대', 'Wi-Fi', '256GB')
('iPad Air', '4세대', 'Wi-Fi+Cellular', '64GB')
('iPad Air', '4세대', 'Wi-Fi+Cellular', '256GB')
('iPad Air', '5세대', 'Wi-Fi', '64GB')
('iPad Air', '5세대', 'Wi-Fi', '256GB')
('iPad Air', '5세대', 'Wi-Fi+Cellular', '64GB')
('iPad Air', '5세대', 'Wi-Fi+Cellular', '256GB')
('iPad', '8세대', 'Wi-Fi', '32GB')
('iPad', '8세대', 'Wi-Fi', '128GB')
('iPad', '8세대', 'Wi-Fi+Cellular', '32GB')
('iPad', '8세대', 'Wi-Fi+Cellular', '128GB')
('iPad', '9세대', 'Wi-Fi', '64GB')
('iPad', '9세대', 'Wi-Fi', '256GB')
('iPad', '9세대', 'Wi-Fi+Cellular', '64GB')
('iPad', '9세대', 'Wi-Fi+Cellular', '256GB')
('iPad', '10세대', 'Wi-Fi', '64GB')
('iPad', '10세대', 'Wi-Fi', '256GB')
('iPad', '10세대', 'Wi-Fi+Cellular', '64GB')
('iPad', '10세대', 'Wi-Fi+Cellular', '256GB')
('iPad Pro 11', '2세대', 'Wi-Fi', 

In [25]:
import datetime

start_date = datetime.datetime(2023, 5, 1)
end_date = datetime.datetime(2023, 5, 31)

datas = []
for model in df["model"].unique().tolist():
    model_data = df[df["model"] == model]
    for screen_size in model_data["screen_size"].unique():
        if screen_size == 11.0:
            screen_size = 11
        size_data = model_data[model_data["screen_size"] == screen_size]
        for gen in size_data["generation"].unique():
            gen_data = size_data[size_data["generation"] == gen]
            for storage in gen_data["storage"].unique():
                storage_data = gen_data[gen_data["storage"] == storage]
                for cellular in [True, False]:
                    for unopened in [True, False]:
                        entity_list = storage_data[(storage_data["cellular"] == cellular) & (storage_data["unused"] == unopened)]

                        key_model = f"{model} {screen_size}" if model == "iPad Pro" else f"{model}"
                        key_gen = f"{gen[:-2]}세대"
                        key_cellular = "Wi-Fi+Cellular" if cellular else "Wi-Fi"
                        key_storage = f"{storage}"

                        key = (key_model, key_gen, key_cellular, key_storage)
                        if key not in api_db:
                            print(f"key not found: {key}")
                            continue

                        item_id, option_id = api_db[key]

                        if entity_list.size <= 1:
                            datas.append({
                                "item_id": item_id,
                                "option_id": option_id,
                                "start_date": start_date.strftime("%Y-%m-%d"),
                                "end_date": end_date.strftime("%Y-%m-%d"),
                                "unopened": unopened,
                                "cellular": cellular,
                                "mean_std": entity_list["price for statistics"].mean() if entity_list.size == 1 else 0,
                                "mean_20": 0,
                                "mean_80": 0,
                            })
                            continue

                        # 전처리
                        entity_list.loc[:, "price for statistics"] = entity_list["price for statistics"]

                        # 평균
                        mean = entity_list["price for statistics"].mean(numeric_only=True)

                        # 중앙 값
                        median = entity_list["price for statistics"].median()

                        # 표준 편차
                        std = entity_list["price for statistics"].std()

                        # 평균으로부터 표준편차 이내의 값 평균
                        mean_std = entity_list[(entity_list["price for statistics"] >= (mean - std)) & (entity_list["price for statistics"] <= (mean + std))]["price for statistics"].mean()

                        # 상위 0 ~ 20% 사이 값들의 평균
                        top_20_percentile = entity_list["price for statistics"].quantile(0.8)
                        top_20_percent_values = entity_list[entity_list["price for statistics"] >= top_20_percentile]
                        mean_20 = top_20_percent_values.mean(numeric_only=True)["price for statistics"]

                        # 하위 20% 평균
                        bottom_20_percentile = entity_list["price for statistics"].quantile(0.2)
                        bottom_20_percent_values = entity_list[entity_list["price for statistics"] <= bottom_20_percentile]
                        mean_80 = bottom_20_percent_values.mean(numeric_only=True)["price for statistics"]

                        if mean_80 == mean_20:
                            mean_std = mean_20
                            mean_80 = mean_20 = 0

                        datas.append({
                            "item_id": item_id,
                            "option_id": option_id,
                            "start_date": start_date.strftime("%Y-%m-%d"),
                            "unopened": unopened,
                            "cellular": cellular,
                            "end_date": end_date.strftime("%Y-%m-%d"),
                            "mean_std": round(mean_std),
                            "mean_20": round(mean_20),
                            "mean_80": round(mean_80),
                        })

# Save to CSV
df = pd.DataFrame(datas)
df.to_csv("ipad-data.csv", index=False)

/tmp/ipykernel_24740/2921639285.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entity_list.loc[:, "price for statistics"] = entity_list["price for statistics"]
/tmp/ipykernel_24740/2921639285.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entity_list.loc[:, "price for statistics"] = entity_list["price for statistics"]
/tmp/ipykernel_24740/2921639285.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

key not found: ('iPad Pro 12.9', '5세대', 'Wi-Fi+Cellular', '2TB')
key not found: ('iPad Pro 12.9', '5세대', 'Wi-Fi+Cellular', '2TB')
key not found: ('iPad Pro 12.9', '5세대', 'Wi-Fi', '2TB')
key not found: ('iPad Pro 12.9', '5세대', 'Wi-Fi', '2TB')


/tmp/ipykernel_24740/2921639285.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entity_list.loc[:, "price for statistics"] = entity_list["price for statistics"]
/tmp/ipykernel_24740/2921639285.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entity_list.loc[:, "price for statistics"] = entity_list["price for statistics"]
/tmp/ipykernel_24740/2921639285.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

In [ ]:
import time
# API Call
api_url_template = "https://asia-northeast3-macguider-44e7e.cloudfunctions.net/api/item/{}/option/{}"

# read csv
df = pd.read_csv("ipad-data.csv")

for index, row in df.iterrows():
    item_id = row["item_id"]
    option_id = row["option_id"]
    date = row["end_date"]
    mean = row["mean_std"]
    high = row["mean_20"]
    low = row["mean_80"]
    unopened = row["unopened"]
    cellular = row["cellular"]


    api_url = api_url_template.format(item_id, option_id)
    if unopened:
        api_url += "?unopened=true"
    else:
        api_url += "?unopened=false"

    print(api_url)

    data = {
        "date": date,
        "mid": int(mean),
        "high": int(high),
        "low": int(low),
    }


    print(f"{data}")
    response = requests.post(api_url, json=data)
    print(response.status_code)
    print(response.text)
    print()
    time.sleep(2)


https://asia-northeast3-macguider-44e7e.cloudfunctions.net/api/item/8/option/11?unopened=true
{'date': '2023-05-31', 'mid': 0, 'high': 0, 'low': 0}
200
{"_writeTime":{"_seconds":1686285556,"_nanoseconds":978537000}}

https://asia-northeast3-macguider-44e7e.cloudfunctions.net/api/item/8/option/11?unopened=false
{'date': '2023-05-31', 'mid': 0, 'high': 0, 'low': 0}
200
{"_writeTime":{"_seconds":1686285561,"_nanoseconds":197548000}}

https://asia-northeast3-macguider-44e7e.cloudfunctions.net/api/item/8/option/9?unopened=true
{'date': '2023-05-31', 'mid': 524667, 'high': 652500, 'low': 505000}
200
{"_writeTime":{"_seconds":1686285563,"_nanoseconds":572959000}}

